In [1]:
import pandas as pd
import scipy.stats as st
import math as m
import statsmodels.stats.api as sm
import numpy as np

In [2]:
df = pd.read_csv("pizza_restaurant.csv")
df.head()

,Product Name,Crust,Toppings,Price,Delivery Time,# pizzas the customer ordered before
0,Pepperoni,Cheese_and_Garlic,4,17,26.3,4
1,Hawaiian,Cheese_and_Chili,4,17,27.8,4
2,Calzone,Cheese_and_Garlic,3,20,31.5,7
3,Margherita,Cheese,4,23,20.8,7
4,Calzone,Cheese_and_Garlic,4,19,27.7,8


In [3]:
df.describe()

,Toppings,Price,Delivery Time,# pizzas the customer ordered before
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,3.965000,19.342000,25.061100,5.659000
std,1.021185,3.345479,2.490397,2.459831
min,1.000000,12.000000,17.800000,0.000000
25%,3.000000,17.000000,23.300000,4.000000
50%,4.000000,19.000000,25.100000,5.000000
75%,5.000000,21.000000,26.700000,7.000000
max,7.000000,33.000000,32.400000,15.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Product Name                          1000 non-null   object 
 1   Crust                                 1000 non-null   object 
 2   Toppings                              1000 non-null   int64  
 3   Price                                 1000 non-null   int64  
 4   Delivery Time                         1000 non-null   float64
 5   # pizzas the customer ordered before  1000 non-null   int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 47.0+ KB


## Standard Error of the Sample Mean

In [5]:
# Calculate the standard error of the Price column

# using the formula SD / sqrt(n)
print(df.Price.std() / m.sqrt(df.Price.count()))
# using scipy's standard error function
print(st.sem(df.Price))

0.105793327900337
0.105793327900337


## Standardization and z-score

In [8]:
# Standardize the 'Delivery Time' column
# calculated as (value - mean) / standard deviation
df['delivery_time_standardized'] = (df['Delivery Time'] - df['Delivery Time'].mean()) / df['Delivery Time'].std()
df['delivery_time_standardized'].head()

0    0.497471
1    1.099784
2    2.585491
3   -1.711012
4    1.059630
Name: delivery_time_standardized, dtype: float64

In [10]:
# Standardize the 'Delivery Time' column using Sklearn
from sklearn import preprocessing

df['delivery_time_standardized2'] = preprocessing.scale(df['Delivery Time'])
df['delivery_time_standardized2'].head()

0    0.497720
1    1.100335
2    2.586785
3   -1.711868
4    1.060160
Name: delivery_time_standardized2, dtype: float64

In [11]:
df.describe()

,Toppings,Price,Delivery Time,# pizzas the customer ordered before,delivery_time_standardized,delivery_time_standardized2
count,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03
mean,3.965000,19.342000,25.061100,5.659000,1.385558e-16,1.421085e-16
std,1.021185,3.345479,2.490397,2.459831,1.000000e+00,1.000500e+00
min,1.000000,12.000000,17.800000,0.000000,-2.915639e+00,-2.917098e+00
25%,3.000000,17.000000,23.300000,4.000000,-7.071562e-01,-7.075101e-01
50%,4.000000,19.000000,25.100000,5.000000,1.562000e-02,1.562781e-02
75%,5.000000,21.000000,26.700000,7.000000,6.580878e-01,6.584171e-01
max,7.000000,33.000000,32.400000,15.000000,2.946879e+00,2.948354e+00
